In [185]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/edsa-movie-recommendation-predict/sample_submission.csv
/kaggle/input/edsa-movie-recommendation-predict/movies.csv
/kaggle/input/edsa-movie-recommendation-predict/imdb_data.csv
/kaggle/input/edsa-movie-recommendation-predict/genome_tags.csv
/kaggle/input/edsa-movie-recommendation-predict/genome_scores.csv
/kaggle/input/edsa-movie-recommendation-predict/train.csv
/kaggle/input/edsa-movie-recommendation-predict/test.csv
/kaggle/input/edsa-movie-recommendation-predict/tags.csv
/kaggle/input/edsa-movie-recommendation-predict/links.csv


In [186]:
# Add packages

# Import our regular old heroes 
import numpy as np
import pandas as pd
import scipy as sp # <-- The sister of Numpy, used in our code for numerical efficientcy. 
import matplotlib.pyplot as plt
import seaborn as sns
import random

# Entity featurization and similarity computation
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.feature_extraction.text import TfidfVectorizer

# Libraries used during sorting procedures.
import operator # <-- Convienient item retrieval during iteration 
import heapq # <-- Efficient sorting of large lists

# Imported for our sanity
import warnings
warnings.filterwarnings('ignore')

In [187]:
# load datasets
df_train = pd.read_csv('/kaggle/input/edsa-movie-recommendation-predict/train.csv')
df_movies = pd.read_csv('/kaggle/input/edsa-movie-recommendation-predict/movies.csv')
df_imdb_data = pd.read_csv('/kaggle/input/edsa-movie-recommendation-predict/imdb_data.csv')
df_genome_tags = pd.read_csv('/kaggle/input/edsa-movie-recommendation-predict/genome_tags.csv')
df_genome_scores = pd.read_csv('/kaggle/input/edsa-movie-recommendation-predict/genome_scores.csv')
df_test = pd.read_csv('/kaggle/input/edsa-movie-recommendation-predict/test.csv')
df_tags = pd.read_csv('/kaggle/input/edsa-movie-recommendation-predict/tags.csv')
df_links = pd.read_csv('/kaggle/input/edsa-movie-recommendation-predict/links.csv')

In [188]:
# get sample of train data due to large size

N_movies  = 10
df_movies = df_movies.iloc[range(0,N_movies),:]

In [189]:
# create dictionary of counts for each table

countdict = {
  "                      Train_Count": df_train.count(),
  "         Movies_Count": df_movies.count(),
  "         Imdb_Count": df_imdb_data.count(),
  "         Genome_Tags_Count": df_genome_tags.count(),
  "         Genome_Scores_Count": df_genome_scores.count(),
  "         Test_Count": df_test.count(),
  "         Tags_Count": df_tags.count(),
  "         Links_Count": df_links.count()
}
print(countdict)

{'                      Train_Count': userId       10000038
movieId      10000038
rating       10000038
timestamp    10000038
dtype: int64, '         Movies_Count': movieId    10
title      10
genres     10
dtype: int64, '         Imdb_Count': movieId          27278
title_cast       17210
director         17404
runtime          15189
budget            7906
plot_keywords    16200
dtype: int64, '         Genome_Tags_Count': tagId    1128
tag      1128
dtype: int64, '         Genome_Scores_Count': movieId      15584448
tagId        15584448
relevance    15584448
dtype: int64, '         Test_Count': userId     5000019
movieId    5000019
dtype: int64, '         Tags_Count': userId       1093360
movieId      1093360
tag          1093344
timestamp    1093360
dtype: int64, '         Links_Count': movieId    62423
imdbId     62423
tmdbId     62316
dtype: int64}


In [190]:
# View heads of dataframe
df_train.head(), df_test.head(),df_movies.head()

(   userId  movieId  rating   timestamp
 0    5163    57669     4.0  1518349992
 1  106343        5     4.5  1206238739
 2  146790     5459     5.0  1076215539
 3  106362    32296     2.0  1423042565
 4    9041      366     3.0   833375837,
    userId  movieId
 0       1     2011
 1       1     4144
 2       1     5767
 3       1     6711
 4       1     7318,
    movieId                               title  \
 0        1                    Toy Story (1995)   
 1        2                      Jumanji (1995)   
 2        3             Grumpier Old Men (1995)   
 3        4            Waiting to Exhale (1995)   
 4        5  Father of the Bride Part II (1995)   
 
                                         genres  
 0  Adventure|Animation|Children|Comedy|Fantasy  
 1                   Adventure|Children|Fantasy  
 2                               Comedy|Romance  
 3                         Comedy|Drama|Romance  
 4                                       Comedy  )

In [191]:
# View heads of dataframe
df_imdb_data.head(), df_genome_tags.head(), df_genome_scores.head()

(   movieId                                         title_cast  \
 0        1  Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...   
 1        2  Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...   
 2        3  Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...   
 3        4  Whitney Houston|Angela Bassett|Loretta Devine|...   
 4        5  Steve Martin|Diane Keaton|Martin Short|Kimberl...   
 
               director  runtime       budget  \
 0        John Lasseter     81.0  $30,000,000   
 1   Jonathan Hensleigh    104.0  $65,000,000   
 2  Mark Steven Johnson    101.0  $25,000,000   
 3       Terry McMillan    124.0  $16,000,000   
 4       Albert Hackett    106.0  $30,000,000   
 
                                        plot_keywords  
 0                   toy|rivalry|cowboy|cgi animation  
 1                   board game|adventurer|fight|game  
 2                         boat|lake|neighbor|rivalry  
 3  black american|husband wife relationship|betra...  
 4                    fath

In [192]:
# View heads of dataframe
df_test.head(), df_tags.head(), df_links.head()

(   userId  movieId
 0       1     2011
 1       1     4144
 2       1     5767
 3       1     6711
 4       1     7318,
    userId  movieId               tag   timestamp
 0       3      260           classic  1439472355
 1       3      260            sci-fi  1439472256
 2       4     1732       dark comedy  1573943598
 3       4     1732    great dialogue  1573943604
 4       4     7569  so bad it's good  1573943455,
    movieId  imdbId   tmdbId
 0        1  114709    862.0
 1        2  113497   8844.0
 2        3  113228  15602.0
 3        4  114885  31357.0
 4        5  113041  11862.0)

In [193]:
# get a subset of the train data
# clean data and create feature space
# dimensionality reduction
# create cosine similarity
# do the predictions
# calculate RMSE


In [194]:
# combine movie data
df_movies_content = pd.merge(df_movies, df_imdb_data, on=['movieId'], how='left')
df_movies_content.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movieId        10 non-null     int64  
 1   title          10 non-null     object 
 2   genres         10 non-null     object 
 3   title_cast     10 non-null     object 
 4   director       10 non-null     object 
 5   runtime        10 non-null     float64
 6   budget         9 non-null      object 
 7   plot_keywords  10 non-null     object 
dtypes: float64(1), int64(1), object(6)
memory usage: 720.0+ bytes


In [195]:
df_movies_content.head(100)

,movieId,title,genres,title_cast,director,runtime,budget,plot_keywords
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Jumanji (1995),Adventure|Children|Fantasy,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Grumpier Old Men (1995),Comedy|Romance,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Father of the Bride Part II (1995),Comedy,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion
5,6,Heat (1995),Action|Crime|Thriller,Al Pacino|Robert De Niro|Val Kilmer|Jon Voight...,Michael Mann,170.0,"$60,000,000",gun battle|organized crime|honor|criminal inve...
6,7,Sabrina (1995),Comedy|Romance,Harrison Ford|Julia Ormond|Greg Kinnear|Nancy ...,Samuel A. Taylor,127.0,"$58,000,000",character name as title|chauffeur|estate|playboy
7,8,Tom and Huck (1995),Adventure|Children,Jonathan Taylor Thomas|Brad Renfro|Eric Schwei...,Mark Twain,97.0,NaN,rescue from drowning|near drowning|character n...
8,9,Sudden Death (1995),Action,Jean-Claude Van Damme|Powers Boothe|Raymond J....,Karen Elise Baldwin,111.0,"$35,000,000",hockey|death in title|two word title|terrorist
9,10,GoldenEye (1995),Action|Adventure|Thriller,Pierce Brosnan|Sean Bean|Izabella Scorupco|Fam...,Ian Fleming,130.0,"$60,000,000",official james bond series|james bond characte...


In [196]:
# initialize 

tokenizer = nltk.RegexpTokenizer(r"\w+")
lemmatizer = WordNetLemmatizer()
stopwords = stopwords.words('english')


In [197]:
nltk.download(['punkt','stopwords'])

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [198]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
#! unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [199]:
# Tokenize, Lemmatize, Stopwords

def text_to_token(s):
    tokens = tokenizer.tokenize(s)
    lower_cased_tokens = [t.lower() for t in tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(t) for t in lower_cased_tokens]
    useful_tokens = [t for t in lemmatized_tokens if t not in stopwords]
    return useful_tokens

In [200]:
def clean_text_two(text):
    #text = str(text)
    text = re.sub('[\d-]', '', text) #remove any numbers
    text = re.sub(r'RT', '', text) #remove the retweets    
    return text

In [201]:
def clean_director(text):
    #lower case
    tokens = tokenizer.tokenize(text)
    lower_cased_tokens = [t.lower() for t in tokens]
    #join on space
    return ''.join(lower_cased_tokens)    

In [202]:
def clean_genres(text):
    
    if text == '(no genres listed)':
        clean_text = ''
    else:
        text = text.split('|')
        clean_text = [t.lower() for t in text]
        clean_text = ' '.join(clean_text)  
    
    return clean_text

In [203]:
def clean_title_cast(text):
    
    text = text.split('|')
    clean_text = [t.lower() for t in text]
    clean_text = [i.replace(" ", "") for i in clean_text]
    clean_text = ' '.join(clean_text)
    
    return clean_text

In [204]:
def remove_punctuation(text):   
    return ''.join([l for l in text if l not in string.punctuation])

In [205]:
# clean title
df_movies_content['title'] = df_movies_content['title'].fillna('')
df_movies_content['title'] = df_movies_content['title'].apply(remove_punctuation)
df_movies_content['title'] = df_movies_content['title'].apply(text_to_token)
df_movies_content['title'] = df_movies_content['title'].apply(lambda x: " ".join(x))

In [206]:
# clean director 

df_movies_content['director'] = df_movies_content['director'].fillna('')
df_movies_content['director'] = df_movies_content['director'].apply(clean_director)


In [207]:
# clean genre 

df_movies_content['genres'] = df_movies_content['genres'].fillna('')
df_movies_content['genres'] = df_movies_content['genres'].apply(clean_genres)


In [208]:
# clean keywords
df_movies_content['plot_keywords'] = df_movies_content['plot_keywords'].fillna('')
df_movies_content['plot_keywords'] = df_movies_content['plot_keywords'].apply(clean_genres)
df_movies_content['plot_keywords'] = df_movies_content['plot_keywords'].apply(text_to_token)
df_movies_content['plot_keywords'] = df_movies_content['plot_keywords'].apply(lambda x: " ".join(x))

In [209]:
# clean title_cast
df_movies_content['title_cast'] = df_movies_content['title_cast'].fillna('')
df_movies_content['title_cast'] = df_movies_content['title_cast'].apply(clean_title_cast)

In [210]:
df_movies_content.tail()

,movieId,title,genres,title_cast,director,runtime,budget,plot_keywords
5,6,heat 1995,action crime thriller,alpacino robertdeniro valkilmer jonvoight toms...,michaelmann,170.0,"$60,000,000",gun battle organized crime honor criminal inve...
6,7,sabrina 1995,comedy romance,harrisonford juliaormond gregkinnear nancymarc...,samuelataylor,127.0,"$58,000,000",character name title chauffeur estate playboy
7,8,tom huck 1995,adventure children,jonathantaylorthomas bradrenfro ericschweig ch...,marktwain,97.0,NaN,rescue drowning near drowning character name t...
8,9,sudden death 1995,action,jean-claudevandamme powersboothe raymondj.barr...,karenelisebaldwin,111.0,"$35,000,000",hockey death title two word title terrorist
9,10,goldeneye 1995,action adventure thriller,piercebrosnan seanbean izabellascorupco famkej...,ianfleming,130.0,"$60,000,000",official james bond series james bond characte...


In [211]:
df_movies_content['movie_soup'] = (pd.Series(df_movies_content[['title','genres','title_cast','director','plot_keywords']]
                                                .fillna('').values.tolist()).str.join(' '))

# indexes to map between movie titles and indexes of the movies dataframe
#titles = df_movies_content['title']
indices = pd.Series(df_movies_content.index, index=df_movies_content['movieId'])

In [293]:
df_movies_content.head()

,movieId,title,genres,title_cast,director,runtime,budget,plot_keywords,movie_soup
0,1,toy story 1995,adventure animation children comedy fantasy,tomhanks timallen donrickles jimvarney wallace...,johnlasseter,81.0,"$30,000,000",toy rivalry cowboy cgi animation,toy story 1995 adventure animation children co...
1,2,jumanji 1995,adventure children fantasy,robinwilliams jonathanhyde kirstendunst bradle...,jonathanhensleigh,104.0,"$65,000,000",board game adventurer fight game,jumanji 1995 adventure children fantasy robinw...
2,3,grumpier old men 1995,comedy romance,waltermatthau jacklemmon sophialoren ann-margr...,markstevenjohnson,101.0,"$25,000,000",boat lake neighbor rivalry,grumpier old men 1995 comedy romance waltermat...
3,4,waiting exhale 1995,comedy drama romance,whitneyhouston angelabassett lorettadevine lel...,terrymcmillan,124.0,"$16,000,000",black american husband wife relationship betra...,waiting exhale 1995 comedy drama romance whitn...
4,5,father bride part ii 1995,comedy,stevemartin dianekeaton martinshort kimberlywi...,alberthackett,106.0,"$30,000,000",fatherhood doberman dog mansion,father bride part ii 1995 comedy stevemartin d...


In [274]:
indices.count()

10

In [281]:
#tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')

from sklearn.feature_extraction.text import CountVectorizer
tf = CountVectorizer(ngram_range = (1,2)) 

# Produce a feature matrix, where each row corresponds to a movie, with vectorized features as columns 
tf_movie_soup_matrix = tf.fit_transform(df_movies_content['movie_soup'])

In [282]:
print (tf_movie_soup_matrix.shape)

(10, 514)


In [ ]:
# add other numerical features here

In [283]:
#PCA
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

svd = TruncatedSVD(n_components = 5)

svd_trunc_tf_movie_soup_matrix = csr_matrix(svd.fit_transform(tf_movie_soup_matrix))

In [284]:
print(svd_trunc_tf_movie_soup_matrix.shape)

(10, 5)


In [285]:
cosine_sim_movie_soup = cosine_similarity(svd_trunc_tf_movie_soup_matrix, svd_trunc_tf_movie_soup_matrix)

print (cosine_sim_movie_soup.shape)

(10, 10)


In [286]:
# Sort train data and limit > 33779
df_train = df_train.sort_values('movieId')
df_train.head()

,userId,movieId,rating,timestamp
7064283,120877,1,3.0,1135874095
412902,35206,1,2.5,1566247848
304411,82765,1,5.0,1308239036
2248784,57880,1,3.0,860324636
6919817,159936,1,4.5,1225209186


In [287]:
# drop rows with movie id > 101
df_train_cropped = df_train[df_train['movieId'] < 11]
df_train_cropped.tail()

,userId,movieId,rating,timestamp
1371848,76627,10,3.0,844086224
4838315,5568,10,5.0,843774293
4681327,84442,10,4.0,1154374997
7519083,73087,10,5.0,841480170
5905508,49049,10,3.0,997905659


In [288]:
# reindex tables

df_users_info = df_train_cropped
df_users_info['userId_movieId'] = df_users_info['userId'].apply(str) + "_" + df_users_info['movieId'].apply(str)

df_users_info = df_users_info.drop(['timestamp'], axis = 1)
df_users_info.tail()


,userId,movieId,rating,userId_movieId
1371848,76627,10,3.0,76627_10
4838315,5568,10,5.0,5568_10
4681327,84442,10,4.0,84442_10
7519083,73087,10,5.0,73087_10
5905508,49049,10,3.0,49049_10


In [289]:
def content_generate_rating_estimate(Comb_Id):
    
    k=2
    threshold=0.9
    
    user = int(Comb_Id.split('_')[0])
    movie_id = int(Comb_Id.split('_')[1])
     
    # Convert the movie id to a numeric index for our similarity matrix
    b_idx = indices[movie_id]
    neighbors = [] # <-- Stores our collection of similarity values 
     
    # Gather the similarity ratings between each movie the user has rated and the reference movie 
    for index, row in df_users_info[df_users_info['userId']==user].iterrows():
        sim = cosine_sim_movie_soup[b_idx-1, indices[row['movieId']]-1]
        neighbors.append((sim, row['rating']))
    
    # Select the top-N values from our collection
    k_neighbors = heapq.nlargest(k, neighbors, key=lambda t: t[0])

    # Compute the weighted average using similarity scores and 
    # user item ratings. 
    simTotal, weightedSum = 0, 0
    for (simScore, rating) in k_neighbors:
        
        # Ensure that similarity ratings are above a given threshold
        if (simScore > threshold):
            simTotal += simScore
            weightedSum += simScore * rating
    try:
        predictedRating = weightedSum / simTotal
    except ZeroDivisionError:
        # Cold-start problem - No ratings given by user. 
        # We use the average rating for the reference item as a proxy in this case 
        predictedRating = np.mean(df_users_info[df_users_info['movieId']==movie_id]['rating'])
    return predictedRating

In [290]:
df_users_info['predictedRating'] = df_users_info['userId_movieId'].apply(content_generate_rating_estimate)

In [291]:
df_users_info.head()

,userId,movieId,rating,userId_movieId,predictedRating
7064283,120877,1,3.0,120877_1,3.0
412902,35206,1,2.5,35206_1,2.5
304411,82765,1,5.0,82765_1,5.0
2248784,57880,1,3.0,57880_1,3.0
6919817,159936,1,4.5,159936_1,4.5


In [292]:
from sklearn.metrics import mean_squared_error

#convert column to numpy array
actual_r = df_users_info['rating'].to_numpy()
predicted_r = df_users_info['predictedRating'].to_numpy()

rmse = np.sqrt(mean_squared_error(actual_r, predicted_r))
rmse

0.049316503757650836

In [ ]:
# run predictions on test data

In [ ]:
# create submission csv
